In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import numpy as np
from nanomesh import Plane
from nanomesh.plane import Plane
import pyvista as pv
from skimage import filters

In [ ]:
plane = Plane.load('x500.npy')
plane = plane.crop(left=150, right=725, top=410, bottom=1470)
plane.show()

In [ ]:
plane_gauss = plane.apply(filters.gaussian, sigma=5)
# filters.try_all_threshold(plane_gauss.image, figsize=(5, 10), verbose=False)

## Local thresholding

### User-defined local threshold

https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_local  
https://scikit-image.org/docs/stable/auto_examples/applications/plot_thresholding.html#local-thresholding

In [ ]:
from skimage.filters import threshold_local
import matplotlib.pyplot as plt

offset = 150
block_size = 101

image = plane_gauss.image

local_thresh = threshold_local(image, block_size, offset=offset)
seg_local = image > local_thresh

### Otsu local threshold

Applies an otsu rank filter to determine the local otsu threshold to segment on.

https://scikit-image.org/docs/stable/api/skimage.filters.rank.html#skimage.filters.rank.otsu  
https://scikit-image.org/docs/stable/auto_examples/applications/plot_rank_filters.html#image-threshold

In [ ]:
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
from skimage.util import img_as_ubyte

image_orig = normalize_values(plane_gauss.image)
image = img_as_ubyte(image_orig)

radius = 41
selem = disk(radius)

local_otsu = rank.otsu(image, selem)
seg_local_otsu = image >= local_otsu

### Adaptive histogram equalization

Using adaptive histogram equalization to remove the gradient. Use standard global thresholds to segment.

https://scikit-image.org/docs/dev/api/skimage.exposure.html#equalize-adapthist  
https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_local_equalize.html#local-histogram-equalization

In [ ]:
from skimage import exposure

def normalize_values(image):
    return (image - image.min()) / (image.max() - image.min())

plane_eq = plane.apply(normalize_values)
plane_eq = plane_eq.apply(exposure.equalize_adapthist)
plane_eq_gauss = plane_eq.apply(filters.gaussian, sigma=5)

# filters.try_all_threshold(plane_eq_gauss.image, figsize=(5, 10), verbose=False)

thresh = filters.threshold_otsu(plane_eq_gauss.image)
seg_clahe = plane_eq_gauss.apply(np.digitize, bins=[thresh])

### Compare results

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(8, 6))
ax = axes.ravel()
plt.gray()

ax[0].imshow(plane.image)
ax[0].set_title('Original')

ax[1].imshow(local_otsu, cmap=plt.cm.gray)
ax[1].set_title('Local Otsu (radius=%d)' % radius)

ax[2].imshow(seg_local_otsu, cmap=plt.cm.gray)
ax[2].set_title('Original >= Local Otsu')

ax[3].imshow(seg_local)
ax[3].set_title('Local thresholding')

ax[4].imshow(plane_eq.image, cmap=plt.cm.gray)
ax[4].set_title('CLAHE')

ax[5].imshow(seg_clahe.image, cmap=plt.cm.gray)
ax[5].set_title('CLAHE+gauss >= otsu')

for a in ax:
    a.axis('off')

plt.show()

## Contour finding

In [ ]:
from skimage.segmentation import clear_border

# select subsection for faster processing
sub_seg = seg_local[20:110, 40:220]

# seg = (~clear_border(~sub_seg)).astype(int)
seg = sub_seg

plt.close('all')
plt.imshow(seg)
plt.show()

In [ ]:
%%time

from nanomesh.mesh2d import Mesher2D

mesher = Mesher2D(seg)

mesher.add_points(label=1, method='gmm', verbose=3)
mesher.add_points(label=0, point_density=0.01, method='gmm', verbose=3)
mesher.generate_contours(max_contour_dist=4)
mesher.generate_edge_contours(max_contour_dist=10)
mesher.generate_mesh()
mesher.generate_domain_mask()

mesher.plot_steps()